# Beispiel Qualitätstest ohne unittest-Framework
In diesem Notebook wird ein Test gezeigt, bei dem nicht auf das unittest-Framework zurückgegriffen wird. Der Test ist sehr ausführlich und entsprechend kommentiert, da er instruktiven Charakter haben soll.

In [1]:
import ifcopenshell
from ifcopenshell.api import run
import ifcopenshell.util.element
import ifcopenshell.util.selector
import re
from collections import Counter

In [2]:
# Laden der IFC-Datei.
model = ifcopenshell.open("../project_data/script_output_4x3.ifc")
model = ifcopenshell.open("../project_data/script_output_4x3_with_errors.ifc")

In [11]:
def Modellaufbau_Borehole_GeotechnicalStratum(model):
    """
    Beschreibung:
        Ein IfcBorehole dient als Container für verschiedene IfcGeotechnicalStratum. IfcBorehole beschreibt die Bohrung als ganzes, wohin IfcGeotechnicalStratum zur Repräsentation von Ansprachebereichen verwendet wird.
    Anweisung:
        Der Modellaufbau ist durch den Fachplaner für Geotechnik sicherzustellen.
    """

    # Filtern der Elemente
    elems = model.by_type("IfcGeotechnicalStratum")
    elems = [i for i in elems if i.ObjectType == "ANSPRACHEBEREICH"]
    
    # Alternativ unter Nutzung der Query Syntax
    elems = ifcopenshell.util.selector.filter_elements(model, "IfcGeotechnicalStratum, ObjectType=ANSPRACHEBEREICH")

    # Für jedes Element: Prüfen der Anforderung
    passed, failed = [], []
    for elem in elems:
        if not elem.Decomposes:
            failed.append(elem)
            continue
        if any((i.RelatingObject.is_a("IfcBorehole") and i.is_a("IfcRelAggregates")) for i in elem.Decomposes): # Hinweis: Hier muss minndestens eine der IfcRelAggregates auf ein IfcBorehole verweisen. Wenn es genau eins sein soll, dann len(elem.Decomposes) == 1 und elem.Decomposes[0] testen
            passed.append(elem)
        else:
            failed.append(elem)
    
    print(f"{len(passed)} Elemente haben bestanden, {len(failed)} sind durchgefallen.")
    if failed:
        print(f"Durchgefallen sind:\n")
        for i in failed:
            print(f"\t{failed}")

    return passed, failed

Modellaufbau_Borehole_GeotechnicalStratum(model)    
pass

15 Elemente haben bestanden, 1 sind durchgefallen.
Durchgefallen sind:

	[#58686=IfcGeotechnicalStratum('0eMnmQ6Qf9u9$Yv2NP_MMS',$,'wrong_borehole_name_xy',$,'ANSPRACHEBEREICH',$,$,$,.USERDEFINED.)]


In [4]:
def WerteBereich_Kohaesion(model, min_val=0, max_val=1_000):
    """
    Prüft, ob die Eigenschaft CohesionBehaviour im Pset_SolidStratumCapacity in einem Wertebereich liegt
    """
    # Filtern der Elemente
    # Hinweis: IfcSimpleProperties können auf mehrere Arten beschrieben werden, die am häufigst verwendete ist IfcPropertySingleValue
    elems = model.by_type("IfcSimpleProperty")
    #elems = model.by_type("IfcPropertySingleValue") + model.by_type("IfcPropertyBoundedValue") + model.by_type("IfcPropertyEnumeratedValue") + model.by_type("IfcPropertyListValue") + model.by_type("IfcPropertyReferenceValue") + model.by_type("IfcPropertyTableValue")

    elems = [i for i in elems if i.Name=="CohesionBehaviour"]
    elems = [i for i in elems if any(j.Name=="Pset_SolidStratumCapacity" for j in i.PartOfPset)]

    # Für jedes Element: Prüfen der Anforderung
    passed, failed = [], []
    for elem in elems:
        value = elem.NominalValue.wrappedValue
        if value>=min_val and value<=max_val: # Hinweis: Im IDS Beispiel werden die beiden Tests gesondert gelistet.
            passed.append(elem)
        else:
            failed.append(elem)

    #    print(elem.NominalValue, elem.Unit)
    #    print(elem.HasConstraints)
    #print(dir(elem.NominalValue))
    #print(elem.NominalValue.wrappedValue)


WerteBereich_Kohaesion(model)


In [5]:
# Each IfcBorehole shall have the Pset_BoreholeCommon
elems = model.by_type("IfcBorehole")
passed, failed = [], []
for elem in elems:
    psets = ifcopenshell.util.element.get_psets(elem)
    if "Pset_BoreholeCommon" in ifcopenshell.util.element.get_psets(elem).keys():
        passed.append(elem)
    else:
        failed.append(elem)

In [6]:
# Get the containers for each IfcBorehole
elems = model.by_type("IfcBorehole")
passed, failed = [], []
for elem in elems:
    container = ifcopenshell.util.element.get_container(elem)
    if container:
        print(f"The Borehole {elem.Name} is located in {container.Name}, {container}")
    else:
        print(f"No container for {elem.Name}")

The Borehole bh_001 is located in Baustelle_Fachsektionstage, #13=IfcSite('06nPK_NG90ew36yl3nhkcF',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_002 is located in Baustelle_Fachsektionstage, #13=IfcSite('06nPK_NG90ew36yl3nhkcF',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_003 is located in Baustelle_Fachsektionstage, #13=IfcSite('06nPK_NG90ew36yl3nhkcF',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_004 is located in Baustelle_Fachsektionstage, #13=IfcSite('06nPK_NG90ew36yl3nhkcF',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_005 is located in Baustelle_Fachsektionstage, #13=IfcSite('06nPK_NG90ew36yl3nhkcF',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
No container for wrong_borehole_name
No container for wrong_borehole_name


In [7]:
# Check naming convention
elems = model.by_type("IfcBorehole")
passed, failed = [], []
for elem in elems:
    if re.match( r'^bh_\d{3}$', elem.Name):
        passed.append(elem)
    else:
        failed.append(elem)

In [8]:
# Unique Names
elems =model.by_type("IfcBorehole")
counter = Counter([i.Name for i in model.by_type("IfcBorehole")])
passed, failed = [], []
for elem in elems:
    if counter[elem.Name]==1:
        passed.append(elem.Name)
    else:
        failed.append(elem.Name)

In [ ]:
# Naming convention Ansprachebereiche
elems =model.by_type("IfcGeotechnicalStratum")
elems = [i for i in elems if i.ObjectType=="ANSPRACHEBEREICH"]
passed, failed = [], []
for elem in elems:
    name = elem.Name
    if not elem.Decomposes:
        failed.append(elem)
        continue
    if not any((i.RelatingObject.is_a("IfcBorehole") and i.is_a("IfcRelAggregates")) for i in elem.Decomposes):
        failed.append(elem)
        continue
    for i in elem.Decomposes:
        if (i.RelatingObject.is_a("IfcBorehole") and i.is_a("IfcRelAggregates")):
            bh_name = i.RelatingObject.Name
            if re.match(fr'^{re.escape(bh_name)}_\d{{3}}$', elem.Name):
                passed.append(elem)
            else:
                failed.append(elem)
            break

['A wrong name', 'wrong_borehole_name_xy']


In [ ]:
# Check distances of boreholes
elems =model.by_type("IfcBorehole")

for i in elems:
    parts =  i.IsDecomposedBy
    if not parts:
        continue
    for j in parts:
        stratums = j.RelatedObjects
        for k in stratums:
            placement = k.ObjectPlacement
            rel_to = placement.PlacementRelTo
            #print(placement.PlacementRelTo)
            #print(placement.RelativePlacement.Location.Coordinates)
            
            c1 = placement.PlacementRelTo.RelativePlacement.Location.Coordinates
            c2 = placement.RelativePlacement.Location.Coordinates
            c3 = (c1[0]+c2[0], c1[1]+c2[1], c1[2]+c2[2])  # das sind die unterkanten
            

            representations = k.Representation.Representations
            representation = [i for i in representations if i.ContextOfItems.ContextIdentifier=="Body"][0]
            # The geometry of a borehole thing only consists 
            if len(representation.Items)!=1:
                raise ValueError(f"To many geometries assigned. Expected 1, got {len(representation.Items)}")
            geom = representation.Items[0]
            
            c4 = (c3[0] + geom.ExtrudedDirection.DirectionRatios[0] * geom.Depth,
                  c3[1] + geom.ExtrudedDirection.DirectionRatios[1] * geom.Depth,
                  c3[2] + geom.ExtrudedDirection.DirectionRatios[2] * geom.Depth)
            print(c4) # Das nun endlich die oberkanten.
            
            # We know that it is a swept solid
            if not geom.is_a("IfcExtrudedAreaSolid"):
                raise ValueError(f"Expected an IfcExtrudedAreaSolid")
            position = geom.Position.Location.Coordinates


            
print(dir(geom.ExtrudedDirection))
geom.Depth

(0.0, 0.0, 4.0)
(0.0, 0.0, 3.0)
(0.0, 0.0, 5.0)
(50.0, 0.0, 2.0)
(50.0, 0.0, 3.0)
(50.0, 0.0, 0.0)
(25.0, 50.0, 6.0)
(25.0, 50.0, 3.0)
(25.0, 50.0, 5.0)
(55.0, 100.0, 4.5)
(55.0, 100.0, 6.0)
(55.0, 100.0, 5.0)
(5.0, 80.0, 6.0)
(5.0, 80.0, 5.0)
(5.0, 80.0, 4.5)
['DirectionRatios', 'LayerAssignment', 'StyledByItem', '__annotations__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rge__', '__rgt__', '__rle__', '__rlt__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attribute_name', 'attribute_type', 'compare', 'file', 'get_info', 'get_info_2', 'id', 'is_a', 'is_entity', 'to_string', 'unwrap_value', 'walk', 'wrap_value']


2.5